# 实验四-整数线性规划

<table style="margin: 0 auto; width: 30%; border-collapse: collapse; border: 1px solid black;" data-id="student-info">  
    <colgroup>  
        <col style="width: 35%;">  
        <col style="width: 65%;">  
    </colgroup>  
    <tr>  
        <td style="border: 1px solid black;">班级</td> <td style="border: 1px solid black;">未填写</td>  
    </tr>  
    <tr>  
        <td style="border: 1px solid black;">学号</td> <td style="border: 1px solid black;">未填写</td>  
    </tr>  
    <tr>  
        <td style="border: 1px solid black;">姓名</td> <td style="border: 1px solid black;">未填写</td>  
    </tr>  
    <tr>  
        <td style="border: 1px solid black;">Email</td> <td style="border: 1px solid black;">未填写</td>  
    </tr>  
</table>

- 什么是整数线性规划问题
- 整数线性规划问题的范例
- 使用`pulp`求解整数线性规划问题

## 实验步骤

1. 打开Vscode,选择"实验一-Python变量.ipynb",在Vscode右上角点击Select Kernel, 然后选择base(Python 3.x.x)。
2. 从上至下按照顺序执行该文件中的Python代码。
3. 阅读实验给出的实验指导和代码范例。
4. 按照实验指导完成实验习题。
5. 在习题后面的单元格执行测试代码，对代码进行测试。
6. 在实验总结部分，使用自己语言完成主观题目1和2的填写。
7. 生成实验报告，并通过腾讯在线收集表提交。(注意所有的习题和习题的测试代码必须要运行一次, 否则无法生成实验报告)

## 实验注意事项

1. 请在指定的地方按照实验指导要求来编写代码。
2. 请按照实验指导要求使用指定的变量名或函数名，不要使用其他的名字。
3. 不要添加任何额外的语句。
4. 不要添加任何额外的代码单元格。
5. 不要在不需要的地方修改作业代码，比如创建额外的变量，修改测试文件中的代码。
6. 实验指导中的`...`表示需要你补充代码的部分，其他部分的代码不用修改。
7. 代码提示中会给出估计的代码行数，例如大约1行代码，估计的代码行数只是一个参考值，实际编写时可能会有出入，请根据实际情况来编写。
8. 请独立完成作业，禁止抄袭，发现抄袭行为成绩记零分

## 什么是整数线性规划问题

整数线性规划(Integer Linear Programming, ILP)是线性规划的一种特殊形式，其主要特点是:

- 目标函数是线性的
- 约束条件也是线性的
- 所有变量必须是整数

与普通线性规划的区别在于： 

- 普通线性规划允许变量取实数值, 决策变量的`cat`参数使用默认值`Continuous`
  - 例如： `x = LpVariable('x', cat='Continuous')`
- 整数线性规划要求变量只能取整数值, 决策变量的`cat`参数为`Binary`或者`Integer`
  - 决策变量只能是整数： `x = LpVariable('x', cat='Integer')`
  - 决策变量只能是0或者1： `x = LpVariable('x', cat='Binary')`

整数线性规划比普通线性规划更难求解，属于NP难问题[什么是NP难问题](https://baike.baidu.com/item/NP%E5%AE%8C%E5%85%A8%E9%97%AE%E9%A2%98)。整数线性规划问题也可以使用`pulp`库来求解，但是求解速度可能会很慢。

## 糖果购买问题（背包问题的变种）

我们之前已经接触过 PuLP 作为求解线性规划的软件包。事实上，PuLP 也能有效解决整数线性规划问题。我们将通过几个案例来介绍如何使用 PuLP 建立和求解整数线性规划问题，其基本流程与解决线性规划问题非常相似，只需要确保将变量声明为整数或二元变量即可。


你在一家糖果店希望购买总重 100 克的糖果。我们需要选择购买的糖果类型（可以将其视为饮食问题的整数线性规划版本）。商店提供 $n$ 种不同的糖果品种 $D_1, \ldots, D_n$，选择时需要满足以下条件：

- 每种糖果 $D_j$ 最多可选 $k_j$ 个</li>
- 单颗 $D_j$ 类糖果的售价为 $p_j$</li>
- 单颗 $D_j$ 类糖果的热量为 $c_j$ 卡路里</li>
- 目标是最小化购买总成本</li>
- 糖果礼盒最多容纳 $N$ 颗糖，必须确保所有糖果能装入礼盒</li>
- 总卡路里摄入需满足最小值 $C_{\min}$ 且不超过最大值 $C_{\max}$</li>


由于必须选择整颗糖果，这个问题属于整数线性规划问题。已知参数包括：
- $n$：糖果种类数
- $(k_1, \ldots, k_n)$：各类糖果的最大可选数量
- $(p_1, \ldots, p_n)$：价格表
- $(c_1, \ldots, c_n)$：单位热量表
- $N$：礼盒容量限制
- $C_{\min}$ 和 $C_{\max}$：卡路里上下限

本问题决策变量为
$$x_1, \ldots, x_n$$ 
其中 $x_i$ 表示选择的第 $i$ 类糖果数量。各变量需满足：
$$ 0 \leq x_i \leq k_i,\ i = 1, \ldots, n $$
总数量约束：
$$ \sum_{i=1}^n x_i \leq N $$
热量约束：
$$ C_{\min} \leq \underbrace{\sum_{i=1}^n c_i x_i}_{\text{总摄入热量}} \leq C_{\max} $$

最终构建的整数线性规划模型为：
$$\begin{array}{rll}
\min& \sum_{j=1}^n p_j x_j & \leftarrow\ \text{最小化购买总成本} \\ 
\mathsf{s.t.} & 0 \leq x_j \leq k_j & \leftarrow\ \text{单类糖果数量限制} \\ 
& \sum_{j=1}^n x_j \leq N & \leftarrow\ \text{礼盒容量限制}\\ 
& C_{\min} \leq \sum_{i=1}^n c_i x_i  \leq C_{\max} & \leftarrow\ \text{热量区间限制} \\ 
& x_1, \ldots, x_n \in \mathbb{Z} & \leftarrow\ \text{整数约束}\\
\end{array}$$

接下来使用`pulp`库来求解整数线性规划问题。

In [ ]:
from pulp import *

def solve_candy_knapsack(n, candy_number_limits, candy_prices, candy_calories, N, Cmin, Cmax):
    assert len(candy_number_limits) == n, 'size mismatch'
    assert len(candy_prices) == n, 'size mismatch for prices'
    assert len(candy_calories) == n, 'size mismatch for list of calories'
    assert N >= 1, 'total number of candies per box must be 1 or more'
    assert Cmin <= Cmax, 'minimum calories is greater than the maximum calories'

    # 目标函数：最小化购买糖果的成本
    prob = LpProblem('Candy Knapsack', LpMinimize)

    # 添加n个决策变量，每个变量代表购买某种糖果的数量，cat参数为'Integer', 表示变量为整数
    dVars = [LpVariable(f'x{i}',lowBound=0, upBound=ki, cat='Integer')
                for (i, ki) in enumerate(candy_number_limits)]

    # 设置目标函数，计算购买糖果的总成本，使用lpSum函数来计算
    prob += lpSum([pj*xj for (pj,xj) in zip(candy_prices, dVars)])

    # 添加约束，所有糖果的数量不超过N
    prob += lpSum(dVars) <= N

    # 计算购买糖果的总卡路里，添加约束，总卡路里在[Cmin, Cmax]之间
    calories_of_candies = lpSum([cj*xj for (cj,xj) in zip(candy_calories, dVars)])
    prob += calories_of_candies <= Cmax
    prob += calories_of_candies >= Cmin

    # 求解问题
    status = prob.solve()

    # 输出结果
    if status == constants.LpStatusInfeasible:
        print('问题无解，无法找到满足约束条件的解')
        return
    elif status == constants.LpStatusUnbounded:
        print('问题无界')
        return
    else:
        assert status == constants.LpStatusOptimal, 'Something went wrong while solving since status is either undefined or unsolved'
        # extract values
        print('成功: 找到了最优解')
        solution_values = [x.varValue for x in dVars]
        for (j, svj) in enumerate(solution_values):
            print(f'\t 糖果种类 # {j}: {svj} 个糖果')

        print(f'总成本: {sum([(pj*svj) for (pj, svj) in zip(candy_prices, solution_values)])}')
        print(f'总的卡路里: {sum([cj*xj for (cj,xj) in zip(candy_calories, solution_values)])}')

In [ ]:
n = 5
candy_number_limits = [10, 12, 10, 11, 10]
candy_prices = [0.2, 0.5, 0.1, 0.4, 0.8]
candy_calories = [25, 12, 22, 14, 33]
solve_candy_knapsack(5, candy_number_limits, candy_prices, candy_calories, 12, 250, 500)

## 习题1

工人指派问题

| 工人 | A   | B   | C   | D   |
|------|-----|-----|-----|-----|
| 甲   | 15  | 18  | 21  | 24  |
| 乙   | 19  | 23  | 22  | 18  |
| 丙   | 26  | 17  | 16  | 19  |
| 丁   | 19  | 21  | 23  | 17  |

问题：  
问分别指派哪名工人去完成哪一项工作（每个工人只能被指定一项工作），可使总的消耗时间为最小？总的消耗时间最少是多少？  

提示1： 该问题需要16个决策变量，4个工人和4个工作，每个决策变量的取值为0或1。可以使用4x4的矩阵来保存这些决策变量。

In [ ]:
from pulp import *

def solve_worker_assignment_lp():

    # 创建线性规划问题
    # problem = LpProblem(...)

    # 创建16个决策变量
    # x =   ...     #  可以使用4x4的矩阵来保存这些决策变量，cat参数为Binary（表示只能为0或者1）

    # 添加目标函数
    # problem += ...

    # 添加约束条件，每个工人只能被分配一项任务，每项任务必须分配一个工人
    # problem += ...

    # 求解问题
    # problem.solve()

    # 输出结果
    # status = ...
    # total_time = ...   # 计算最小的消耗时间

    ## 你编写的代码从这里开始




    ## 你编写的代码到这里结束

    result = {
        'status': status,
        'total_time': total_time
    }
    return result

In [ ]:
from testset4 import test_solve_worker_assignment_lp
test_solve_worker_assignment_lp(solve_worker_assignment_lp)

## 习题2

集合覆盖问题

---

假设你正在计划一个公司派对，有 $n$ 个人按 $1, \ldots, n$ 编号，这些人都可能被邀请。我们希望邀请至少 $m$ 个人参加。因此，人力资源部要求你遵守以下限制：

有多个员工团队 $T_1, \ldots, T_k$，其中 $T_i \subseteq \{1, \ldots, n\}$ 是员工的子集。一个员工可以属于多个团队。我们希望确保每个团队至少有 1/4 的成员被邀请。注意，同一个人可以作为多个团队的代表被邀请。

我们还有一组对 $G_1, \ldots, G_l$，其中每个 $G_j$ 由一组有互相争议的员工组成。为了保持派对的欢乐氛围，我们不能从每个争议集 $G_j$ 中邀请超过一名员工。

人力资源部门通过其机器学习算法为每位员工 $i$ 分配了一个"扫兴"分数 $s_i$，分数越高，他们就越可能扫兴。我们希望最小化所有被邀请员工的总扫兴分数。

决策变量：我们将有决策变量 $w_1, \ldots, w_n$，其中 $w_i = 1$ 表示员工 $i$ 被邀请参加派对，$w_i = 0$ 表示他们未被邀请。

目标函数：$\min\ \sum_{i=1}^n s_i w_i$，其中 $s_i$ 是员工 $i$ 的"扫兴"分数。

约束条件：我们将有以下约束：

- $\sum_{j \in T_i} w_j \geq |T_i|/4\ \text{对每个}\ T_i$ (每个团队至少四分之一的成员)
- $\sum_{j \in G_i} w_j \leq 1\ \text{对每个}\ G_i$ (每个争议集最多一人)
- $\sum_{j=1}^n w_j \geq m$ (总共至少邀请 $m$ 人)
- $w_j \in \{0, 1\}$ 对所有 $j \in \{1, \ldots, n\}$ (二元变量)


In [ ]:
from pulp import *
def plan_invite_list(n, m, T_lists, G_lists, pp_scores):
    assert m >= 0, '邀请的员工人数不能是负数'
    assert all( 0 <= j < n for ti in T_lists for j in ti )
    assert all( 0<= j < n for gi in G_lists for j in gi)
    assert len(pp_scores) == n, '扫兴分数列表的长度不等于员工数'

    # 创建线性规划问题
    # problem = LpProblem(...)

    # 创建n个决策变量，存放在一个list中，0表示该索引序号的员工不被邀请，1表示该索引序列的员工被邀请
    # dvars =   [LpVariable(...) for i in range(n) ]

    # 添加目标函数，计算总的扫兴分数
    # problem += ...

    # 添加约束条件，每个小组至少1/4的员工被邀请，每个争议集合最多邀请一个人，总共需要邀请m个人
    # problem += ...
    # problem += ...
    # problem += ...

    # 求解问题
    # problem.solve()

    # 输出结果
    # status = ...
    # sol = ...   # 返回由0和1组成的列表，0表示该索引序号的员工不被邀请，1表示该索引序列的员工被邀请
    # optimal_pop_score = ...    # 返回问题的目标函数的值，也就是最优的扫兴分数

    ## 你编写的代码从这里开始


    ## 你编写的代码到这里结束


    if status == constants.LpStatusInfeasible:  # 问题无解
        return status, [], -1
    elif status != constants.LpStatusOptimal:   # 问题是无界的，没有最优解
        return status, [], -1
    else:                       # 问题有最优解
        # extract values
        sol = [x.varValue for x in dvars]
        return status, sol, optimal_pop_score


In [ ]:
from testset4 import test_plan_invite_list
test_plan_invite_list(plan_invite_list)

## 习题3

在这个问题中，你将把三着色问题设置并求解为整数线性规划问题。

三着色问题输入一个无向图 $G$，其顶点集为 $V = \{1, \ldots, n\}$ 且有无向边集 $E$。我们要寻找一种方法，将每个顶点染成红色、绿色或蓝色三种颜色之一，使得任意边 $(i,j)$ 的两个端点 $i,j$ 具有不同的颜色。

给定一个图，我们希望知道是否存在一个三着色方案，如果存在，我们希望找到这个三着色方案。虽然这看起来像是一个玩具问题，但它在资源分配和其他领域都有实际应用。

首先，我们要求你将三着色问题设置为整数线性规划问题。

决策变量：对于每个顶点 $i \in V$，我们将使用三个决策变量 $x_i^R, x_i^G$ 和 $x_i^B$ 来分别表示该顶点是否被染成红色、绿色或蓝色。注意，这些都是取值为 $0,1$ 的二进制变量。

### (A) 每个顶点只能取一种颜色

用 $x_i^R, x_i^G$ 和 $x_i^B$ 写出一个约束条件（？是你需要填写的内容），表示每个顶点必须恰好被染成红色、绿色或蓝色三种颜色之一。

$x_i^R + x_i^G + x_i^B = ?$

### (B) 相邻顶点不能具有相同的颜色

对于每条边 $(i,j) \in E$，写出表示它们不能具有相同颜色的约束条件。**提示**: 写出三个约束条件，分别表示顶点 $i,j$ 不能同时为绿色、不能同时为红色以及不能同时为蓝色。将这些要求转换为涉及 $x_i^G, x_j^G$， $x_i^R, x_j^R$ 和 $x_i^B, x_j^B$ 的约束条件。


约束条件一：
$$
x_i^G + x_j^G <= ? 
$$

再写两个约束条件：
$$
???  \\
???
$$

编写一个函数，该函数接受顶点数量 $n \geq 1$ 和边列表（以顶点对的列表形式给出）`[ (i1, j1), (i2, j2) ..., (im, jm) ]` 作为输入，返回一个元组 `(flag, color_assignment)`，其中包含布尔值 `flag` 和列表 `color_assignment`，具体要求如下：

- `flag` 如果图是可三着色的则为 `True`，否则为 `False`。
- `color_assignment` 是一个包含 n 个颜色的列表，颜色用 `r`、`g` 或 `b`（分别代表红色、绿色或蓝色）表示，列表中第 $i$ 个元素表示分配给顶点 $i$ 的颜色。

注意，如果 `flag` 设置为 `False`，则返回值中的 `color_assignment` 部分将被忽略。


In [ ]:
from pulp import *

def encode_and_solve_three_coloring(n, edge_list):
    assert n >= 1, 'Graph must have at least one vertex'
    assert all(0 <= i and i < n and 0 <= j and j < n and i != j for (i, j) in edge_list), 'Edge list is not well formed'

    # 创建线性规划问题
    # problem = LpProblem(...)

    # 创建三个长度为n个决策变量list，reds列表中的1表示该索引节点选中的颜色为红色，0表示不是红色，
    # green和blues列表也是类似的意思
    # reds = [LpVariable(...) for i in range(n)]
    # greens = ...
    # blues = ...

    # 添加目标函数，计算总的扫兴分数
    # problem += ...

    # 添加约束条件，每个节点只能选中一种颜色
    # problem += ...

    ## 遍历边列表，添加约束，相邻节点不可以是相同颜色
    # for i, j in edge_list:
    #   problem += ...
    #   problem += ...
    #   problem += ...


    ## 求解问题
    # status = problem.solve()

    # 输出结果，如果有最优解，返回True和颜色列表,颜色列表应该只包含'r','g',或者'b'
    # if status == LpStatusOptimal:
    #     color_assignment = []
    #     for i in range(n):
    #
    #
    #
    #
    #
    #

    #     return (True, color_assignment)
    ## 如果没有解，返回False和[]
    # else:
    #     return (False, [])


    ## 你编写的代码从这里开始


    ## 你编写的代码到这里结束

    prob = ...

    # 1. Formulate the decision variables
    reds = ...
    greens = ...
    blues = ...

    for i in range(n):
        ...


    for i, j in edge_list:
        ...

    status = ...

    # 4. Return the result in the required form to pass the tests
    if status == LpStatusOptimal:
        ...

        return (True, color_assignment)
    else:
        return (False, [])

In [ ]:
from testset4 import test_encode_and_solve_three_coloring
test_encode_and_solve_three_coloring(encode_and_solve_three_coloring)

## 实验自动评分

运行下面的代码进行实验自动评分

In [ ]:
from testset4 import grade_all_tests
grade_all_tests()

## 实验总结

请尽量使用自己的语言回答下面的问题。

问题一： 整数线性规划问题和一般线性规划问题的区别在哪里？为什么解决整数规划问题会比普通线性规划问题慢？

问题2： 整数线性规划问题是NP难问题，什么是NP难问题？查阅资料，有哪些方法可以解决整数线性规划问题？

## 生成实验报告

In [ ]:
import sys
import os

# Add the root directory (parent directory of `util`) to sys.path
root_dir = os.path.abspath("..")  # Adjust based on the structure
sys.path.append(root_dir)

from util import notebook2pdf, notebook_info_extractor

notebook_file = "实验4-整数线性规划.ipynb"
stu_info  = notebook_info_extractor.extract_from_ipynb(notebook_file)

html_file = "notebook.html"
pdf_file = f"{stu_info['class_id']}-{stu_info['student_id']}-{stu_info['name']}-实验报告4.pdf"

notebook2pdf.notebook_to_html(notebook_file, html_file)
notebook2pdf.html_to_pdf(html_file, pdf_file)

Notebook successfully converted to HTML with table styles: notebook.html
Successfully converted notebook.html to 未填写-未填写-未填写.pdf
